Некоторые задания в этой тетрадки были созданы на основе соотвествующей тетрадки курса NLP от Elena Voita.

# Генерация текста: н-граммы

В этой тетрадке мы научимся делать простую модель генерации текста на основе н-грамм и встречаемости в корпусе.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [2]:
random.seed(1)
np.random.seed(1)

Будем пробовать генерировать шутки. Для обучения будем использовать [датасет с постами reddit](https://kaggle.com/datasets/thedevastator/one-million-reddit-jokes).

In [3]:
path = '/content/drive/MyDrive/NLP Course Tutoring/datasets/reddit_jokes.csv'
data = pd.read_csv(path)

In [4]:
data.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,ftbp1i,2qh72,jokes,False,1585785543,https://old.reddit.com/r/Jokes/comments/ftbp1i...,self.jokes,NaN,My corona is covered with foreskin so it is no...,I am soooo glad I'm not circumcised!,2
1,post,ftboup,2qh72,jokes,False,1585785522,https://old.reddit.com/r/Jokes/comments/ftboup...,self.jokes,NaN,It's called Google Sheets.,Did you know Google now has a platform for rec...,9
2,post,ftbopj,2qh72,jokes,False,1585785508,https://old.reddit.com/r/Jokes/comments/ftbopj...,self.jokes,NaN,The vacuum doesn't snore after sex.\r\n\r\n&am...,What is the difference between my wife and my ...,15
3,post,ftbnxh,2qh72,jokes,False,1585785428,https://old.reddit.com/r/Jokes/comments/ftbnxh...,self.jokes,NaN,[removed],My last joke for now.,9
4,post,ftbjpg,2qh72,jokes,False,1585785009,https://old.reddit.com/r/Jokes/comments/ftbjpg...,self.jokes,NaN,[removed],The Nintendo 64 turns 18 this week...,134


Так как наша задача генерации требует только текста, оставим только соответствующий столбец.

In [5]:
columns = ['selftext']
data = data[columns]

## Обработка данных
###1. Чистка датасета
Для начала нужно определиться, есть в данных пропуски, и избавиться от них, если есть.

__Подсказка:__ Часто пропуски они обозначаются как nan, но иногда можно заметить иные способы.

In [6]:
data['selftext'].value_counts()[:10]

,count
selftext,
[removed],232919
[deleted],188442
\[removed\],272
To get to the other side.,125
Dr. Dre,111
A stick.,83
None.,81
A stick,76
He worked it out with a pencil.,74


Можно заметить, что наиболее частым классом являются _removed_ или _deleted_.

In [7]:
print('Размер данных до чистки', data.shape)
data = data[~data.isin(['[removed]', '[deleted]', '\[removed\]', 'removed', 'deleted'])]
data = data.dropna()
print('Размер данных после чистки', data.shape)

Размер данных до чистки (999998, 1)
Размер данных после чистки (573847, 1)


In [8]:
data = data.sample(100000)

Надо тексты привести к нижнему регистру и убрать пунктуацию.
Кроме этого можно избавиться от совсем коротких шуток, так как скорее всего это просто ответы на фразы. При делении текста на слова, используйте word_tokenize.

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Надо тексты привести к нижнему регистру. Пунктуацию можно оставить, так как она влияет на смысл предложения и на встречаемость.
Кроме этого можно избавиться от совсем коротких шуток, так как скорее всего это просто ответы на фразы. При делении текста на слова, используйте word_tokenize.

In [10]:
from tqdm import tqdm
tqdm.pandas()

In [11]:
from string import punctuation
import re
from nltk.tokenize import word_tokenize

In [12]:
def clean_text(text):
    '''
    Делит текст на слова и пунктуацию и приводит все к нижнему регистру
    :param text: строка
    :returns: список слов и знаков препинания
    '''
    text = text.lower()
    new_text = []
    for word in word_tokenize(text):
        new_text.append(word)
    return new_text

In [13]:
data['words'] = data['selftext'].progress_apply(clean_text)
data['lens'] = data['words'].progress_apply(len)
data = data[data.lens > 3]

100%|██████████| 100000/100000 [00:00<00:00, 768943.81it/s]


In [14]:
words = data['words'].tolist()

In [15]:
words[:2]

[['but',
  'recently',
  'i',
  "'ve",
  'build',
  'up',
  'a',
  'tolerance',
  'to',
  'it',
  '.'],
 ['that', 'place', 'is', 'one', 'giant', 'pyramid', 'scheme', '!']]

## N-grams
Для начала попробуем создать самую простую модель, основанную на встречаемости н-граммы в корпусе.

In [16]:
from collections import defaultdict, Counter

In [17]:
# добавляем токены начала и конца
BOS, EOS = '[bos]', '[eos]'

class NGramLanguageModel:
    def __init__(self, lines, n):
        assert n >= 1
        self.n = n
        self.counts = self.ngram_counts(lines, self.n)

        # перевести количества в вероятности
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        for key, value in self.counts.items():
            sum_of_prefix = sum(value.values())
            for word, cnts in value.items():
                self.probs[key][word] = cnts / sum_of_prefix

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: строка запроса
        :returns: словарь с возможными продолжениями заданного префикса
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ BOS ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    @staticmethod
    def ngram_counts(lines, n):
        dictionary = defaultdict(Counter)
        for line in lines:
            new_line = [BOS] * (n-1) + line + [EOS]
            for i in range(n-1, len(new_line)):
                prefix = tuple(new_line[i-n+1:i])
                word = new_line[i]
                dictionary[prefix][word] += 1
        return dictionary

# Проверим работу функции ngram_counts
dummy_lines = sorted(words, key=len)[:100]
dummy_counts = NGramLanguageModel.ngram_counts(dummy_lines, n=3)
# assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
# assert len(dummy_counts[(BOS, BOS)]) == 59
# assert dummy_counts[BOS, 'a']['melon'] == 1

# Проверим работу модели
dummy_lm = NGramLanguageModel(dummy_lines, n=3)
p_initial = dummy_lm.get_possible_next_tokens('')
# assert p_initial.most_common(1)[0][0] == 'a'

In [18]:
lm = NGramLanguageModel(words, n=3)

__Вопрос:__
- видете ли вы, какие недочеты могут быть в такой версии модели?

### Методы составления предложений

__1. Жадный метод__

Берём самое часто встречаемое слово

In [19]:
def get_next_word_greedy(lm, prefix):
    '''
    :param lm: language model
    :param prefix: строка префикса
    :returns: следующее, наиболее вероятное, слово для данного префикса
    '''
    return lm.get_possible_next_tokens(prefix).most_common(1)[0][0]

In [20]:
prefix = 'get'
repeat = 20
for _ in range(repeat):
    word = get_next_word_greedy(lm, prefix)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix)

get off the plane . the man says , `` i 'm not sure if it 's a little bit of


In [21]:
prefix = ''
word = get_next_word_greedy(lm, prefix)
repeat = 20
for _ in range(repeat):
    word = get_next_word_greedy(lm, prefix)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix + f'{word} ')

 i 'm not sure if it 's a little bit of a sudden , a man walks into a barbar 


2. Выбор наиболее вероятного слова не показал хороших результатов. Тем более, он будет строить очень много похожих предложений, а нам хочется разнообразия. Давайте попробуем семплировать методом top-k: выбираем k наиболее встречаемых вариантов и из них выбираем один случайным образом.

In [22]:
def get_next_word_topk(lm: NGramLanguageModel, prefix: str, k:int) -> str:
    '''
    :param lm: language model
    :param prefix: строка префикса
    :param k: количество слов в top-k
    :returns: следующее, наиболее вероятное, слово для данного префикса
    '''
    next_words = lm.get_possible_next_tokens(prefix).most_common(k)
    index = random.randint(0, min(k, len(next_words))-1)
    return next_words[index][0]

In [23]:
# get_next_word_topk(lm, prefix, 5)

In [24]:
prefix = 'want'
repeat = 20
for i in range(repeat):
    word = get_next_word_topk(lm, prefix, 5)
    prefix += ' ' + word
    if prefix.endswith(EOS):
        break

print(prefix)

want to have a wife . the first time i 'm not saying that they have been a bit . the


In [25]:
prefix = ''
word = get_next_word_topk(lm, '', 5)
while word != EOS:
    prefix += f'{word} '
    word = get_next_word_topk(lm, prefix, 5)
print(prefix + f'{word} ')

because they have a good time for the next morning . [eos] 


3. Для сравнения можно сделать beam search. Но это не самое приятное развлечени, поэтому мы этого делать не будем. Но если это надо, то это уже написано, например, в генеративных моделях библиотеки transformers.

## Оценивание модели
Раз мы научились делать простую языковую модель, надо понять, насколько она хорошо описывает язык. Для оценивания этого используем перплексию.

$PPL(W) = 2^{log_{2}(P(W)^{-\frac{1}{N}})} =2^{{-\frac{1}{N}}log_{2}(P(W))}$



In [26]:
data['words'].shape

(86130,)

1. Для начала нужно разделить данные на тренировочные и тестовые

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(data['words'], train_size=0.8)

2. Давайте напишем функцию, которая будет считать перплексию для каждого отдельного предложения на основе n-грамм. Для каждой нграммы считается её вероятность. Нам её считать не надо, так как она уже записана в атрибуте _self.probs_ в нашем классе модели.

In [28]:
def perplexity(model: NGramLanguageModel, text: list, n=1) -> float:
    """
    :param model: language model
    :param text: список н-грамм предложения
    :param n: количество слов в н-грамме
    :returns: значение перплексии одного предложения
    """
    result = 0
    length = 0
    for ngram in text:
        if n == 1:
            prefix = ()
        else:
            prefix = ngram[:n-1]
        prob = model.probs[prefix][ngram[-1]]
        if prob != 0:
            result += np.log2(prob)
        length += 1
    enthropy = -(result / length)
    return 2**enthropy

3. Теперь можем посчитать среднюю перплексию по всему датасету.

In [29]:
from nltk.util import ngrams

In [30]:
n = 1
lm = NGramLanguageModel(X_train, n=n)
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=n))
    value = perplexity(lm, test_sent, n=n)
    avg_ppl += value

avg_ppl /=  len(X_test)
# assert np.isclose(avg_ppl, 1857.90, atol=1e-1)
print(avg_ppl)

1127.897092054018


Попробуем теперь сравнить перплексию для моделей униграм, биграм и триграм. Почему результаты получились такие?

In [31]:
lm = NGramLanguageModel(X_train, n=2)
all_ppls = []
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=2))
    value = perplexity(lm, test_sent, 2)
    all_ppls.append((' '.join(sentence), value))
    avg_ppl += value
print(avg_ppl / len(X_test))

53.79759688020874


In [32]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

('an anti-climactic climatic joke', 1.0) ('the information is groundbreaking', 3357.0038199694604)


In [33]:
lm = NGramLanguageModel(X_train, n=3)
all_ppls = []
avg_ppl = 0
for sentence in X_test:
    test_sent = list(ngrams(sentence, n=3))
    value = perplexity(lm, test_sent, 3)
    all_ppls.append((' '.join(sentence), value))
    avg_ppl += value
print(avg_ppl / len(X_test))

6.199512956583058


In [34]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

('but men can fake love .', 1.0) ('to the dock !', 403.60128840230425)


## Использование готовых инструментов
К счастью, нам всё писать необязательно. Необходимые функции и модули уже были написаны умными программистами. Давайте посмотрим на модуль nltk и на то, что он нам предлагает.

In [35]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

n = 1
train_data, padded_vocab = padded_everygram_pipeline(n, X_train)
model = MLE(n)
model.fit(train_data, padded_vocab)

In [36]:
test_data, _ = padded_everygram_pipeline(n, X_test)
X_test_list = X_test.tolist()
all_ppls = []
avg_ppl = 0
length = 0
for i, test in enumerate(test_data):
    value = model.perplexity(test)
    all_ppls.append((' '.join(X_test_list[i]), value))
    if value != np.inf:
        avg_ppl += value
print(avg_ppl / i)

987.0364336249156


Давайте повторим то же самое но с моделями на биграммах и триграммах. Сравните результаты. Почему они такие?

In [37]:
n = 2
train_data, padded_vocab = padded_everygram_pipeline(n, X_train)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_data, _ = padded_everygram_pipeline(n, X_test)
all_ppls = []
avg_ppl = 0
length = 0
for i, test in enumerate(test_data):
    value = model.perplexity(test)
    all_ppls.append((' '.join(X_test_list[i]), value))
    if value != np.inf:
        avg_ppl += value
print(avg_ppl / i)

45.36418524879905


In [38]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

("`` what ? ''", 33.18575945912123) ("there once was a man from nantucket . his dick was so long he could suck it . he said with a grin , as he wiped from his chin , `` if my ear was a cunt i would fuck it . ''", inf)


In [39]:
n = 3
train_data, padded_vocab = padded_everygram_pipeline(n, X_train)
model = MLE(n)
model.fit(train_data, padded_vocab)

test_data, _ = padded_everygram_pipeline(n, X_test)
all_ppls = []
avg_ppl = 0
length = 0
for i, test in enumerate(test_data):
    value = model.perplexity(test)
    all_ppls.append((' '.join(X_test_list[i]), value))
    if value != np.inf:
        avg_ppl += value
print(avg_ppl / i)

4.915578394055283


In [40]:
s = sorted(all_ppls, key=lambda x: x[1])
print(s[0], s[-1])

("`` what ? ''", 16.107730324857563) ('oh , snap !', inf)


И давайте напоследок попробуем сгенерировать последовательность с помощью обученной модели:

In [41]:
model.generate(20, text_seed=['he'])

['stopped',
 'posting',
 'youtube',
 'videos',
 'and',
 'doing',
 'nothing',
 '.',
 'it',
 'sees',
 'them',
 'all',
 '?',
 "''",
 'says',
 'the',
 'shopkeeper',
 'then',
 'recommends',
 'a']